# Что тут?

Это мой бейзлайн. 

Идея простейшая - имеем пары "запрос - поисковая выдача" и требуется проставить оценку релевантности в этой паре. Так дадим это сделать llm и она, как человек, просмотрит все экземпляры и проставит свою оценку. 

Реализация - цикл по всем экземплярам. Минимальная постобработка исходных данных в промпт и запрос к модели. 

# Импорты

In [1]:
import pandas as pd
import json
from openai import OpenAI
import numpy as np

## Функции и классы

### Prompt 1



```json
            {"role": "system", "content": f"""Ты - эксперт по анализу запросов пользователей для поиска мест на карте.
Твоя задача - определить соответствие запроса пользователя варианту ответа.

Reasoning: low

Шкала оценивания:

0 - Вариант ответа совершенно противоречит запросу пользователя.
1 - Во всех остальных случаях.
    
Вариант ответа:

{answer}

В ОТВЕТ НАПИШИ ТОЛЬКО ОДНУ ИЗ ОЦЕНОК: 0 или 1!"""
    },
```


### Prompt 2


```json
            {"role": "system", "content": f"""Ты - эксперт по анализу запросов пользователей для поиска мест на карте.
Твоя задача - определить соответствие запроса пользователя варианту ответа.

Шкала оценивания:

0 - Вариант ответа совершенно не удовлетворяет или противоречит запросу пользователя.
1 - Вариант ответа частично или полностью удовлетворяет запросу пользователя или приблизительно соответсвует тематике запроса.
    
Вариант ответа:

{answer}

В ОТВЕТ НАПИШИ ТОЛЬКО ОДНУ ИЗ ОЦЕНОК: 0 или 1!"""
    },
```


### Prompt 3

```json
            {"role": "system", "content": f"""Ты - эксперт по анализу запросов пользователей для поиска мест на карте.
Твоя задача - определить соответствие запроса пользователя варианту ответа.

Шкала оценивания:

0.0 - Вариант ответа совершенно не удовлетворяет или противоречит запросу пользователя.
0.1 - Вариант ответа частично удовлетворяет запросу пользователя или приблизительно соответсвует тематике запроса.
1.0 - Вариант ответа полностью удовлетворяет запросу пользователя.
                 
Вариант ответа:

{answer}

В ОТВЕТ НАПИШИ ТОЛЬКО ОДНУ ИЗ ОЦЕНОК: 0 или 1!"""
    },
```

### Функция проставления оценки

In [ ]:
def match_score(query, answer)->dict:
    response = client.chat.completions.create(
        model="gpt-oss:20b",
        messages=[
            {"role": "system", "content": f"""Ты - эксперт по анализу запросов пользователей для поиска мест на карте.
Твоя задача - определить соответствие запроса пользователя варианту ответа.

Шкала оценивания:

0 - Вариант ответа совершенно не удовлетворяет или противоречит запросу пользователя.
1 - Вариант ответа частично или полностью удовлетворяет запросу пользователя или приблизительно соответсвует тематике запроса.
    
Вариант ответа:

{answer}

В ОТВЕТ НАПИШИ ТОЛЬКО ОДНУ ИЗ ОЦЕНОК: 0 или 1!"""
    },
            {"role": "user", "content": query}
        ]
    )
    try:
        data = float(response.choices[0].message.content)
    except:
        data = 0.0
        print(response.choices[0].message.content)
    return data

# Импорт данных

In [ ]:
train_data = pd.read_json(path_or_buf="content/data_final_for_dls_new.jsonl", lines=True)
eval_data = pd.read_json(path_or_buf="content/data_final_for_dls_eval_new.jsonl", lines=True)
eval_data = eval_data[eval_data["relevance"] != 0.1]
eval_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...,0.1
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0
...,...,...,...,...,...,...,...,...,...
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...,0.0
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None,0.0
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...,0.0
568,итальянская пиццерия в санкт-петербурге,"Санкт-Петербург, Капитанская улица, 4",Joy's Pizza; Джойс Пицца; JOY'S PIZZA; Papa Jo...,Пиццерия,133671701112,Joy's Pizza предлагает широкий ассортимент пиц...,0.1,Joy's Pizza — пиццерия с смешанными отзывами: ...,0.1


# BASELINE

Локально гоняю на олламе, чтоб ни в чем себе не отказывать🙃

In [10]:
client = OpenAI(
    base_url="http://localhost:11434/v1",  # Local Ollama API
    api_key="ollama"                       # Dummy key
)

Просто цикл, да вот так💁

In [11]:
from tqdm.notebook import tqdm

predicted_relevance = []
for i, row in tqdm(eval_data.iterrows(), total=len(eval_data), desc="Processing rows"):
    answer = f"""
    Адрес: 
    {row['address']}

    Имя: 
    {row['name']}

    Рубрика: 
    {row['normalized_main_rubric_name_ru']}

    Описание: 
    {row['prices_summarized']}

    Отзывы:
    {row['reviews_summarized']}
    """
    predicted_relevance.append(match_score(row['Text'], answer))

eval_data['predicted_relevance'] = predicted_relevance

Processing rows:   0%|          | 0/570 [00:00<?, ?it/s]

In [12]:
eval_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new,predicted_relevance
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0,0.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0,0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0,1.0
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...,0.1,0.0
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...,0.0,0.0
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None,0.0,0.0
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...,0.0,0.0
568,итальянская пиццерия в санкт-петербурге,"Санкт-Петербург, Капитанская улица, 4",Joy's Pizza; Джойс Пицца; JOY'S PIZZA; Papa Jo...,Пиццерия,133671701112,Joy's Pizza предлагает широкий ассортимент пиц...,0.1,Joy's Pizza — пиццерия с смешанными отзывами: ...,0.1,0.0


Тут я подглядываю в различия на эвал, потому что интересно 👉👈

In [13]:
check = eval_data.loc[eval_data['relevance_new']!=eval_data['predicted_relevance']]

In [ ]:
check_data = pd.read_excel('eval_data.xlsx')

In [14]:
check

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new,predicted_relevance
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0,0.0
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...,0.1,0.0
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0,0.0
8,Купить торт,"Санкт-Петербург, Лиговский проспект, 30",Фабрика Счастье; Lavka schastya; Лавка Счастья...,Кондитерская,1552298395,"Фабрика Счастье предлагает наборы конфет, прал...",1.0,Организация «Фабрика Счастье» занимается прода...,1.0,0.0
10,где в Астрахани сделать фото радужки глаза,"Владимир, Большая Московская улица, 16",Фото радужки глаза; ScreenEyes,Фотоуслуги,238310724487,"Предлагает услуги фотосъёмки, изготовление фот...",1.0,Организация занимается созданием фотографий ра...,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,236976975812,Салон красоты «Сила» предлагает услуги по уход...,1.0,Организация «Сила» занимается предоставлением ...,1.0,0.0
562,Страховой магазин,"Московская область, Балашиха, шоссе Энтузиасто...",Страховой Дом ВСК; Strakhovoy Dom Vsk; Vsk; ВС...,Страховая компания,1092798483,Страховой Дом ВСК предлагает различные страхов...,0.1,Организация занимается страхованием автомобиле...,0.1,1.0
563,фгдс под наркозом,"Санкт-Петербург, улица Олеко Дундича, 36, корп. 2",Консультативно-диагностический центр для детей...,Диагностический центр,1146773851,Консультативно-диагностический центр для детей...,0.1,Организация занимается консультациями и диагно...,0.1,0.0
564,где недорого поесть в парке горького,"Москва, Центральный административный округ, Фо...",Stardogs,Быстрое питание,142948404966,None,0.1,Организация Stardogs занимается быстрым питани...,0.1,0.0


Ну и метрики считаем 🧑‍💻

In [15]:
from sklearn.metrics import accuracy_score, f1_score


# 2. Приводим оба столбца к целочисленному типу (на всякий случай)
y_true = eval_data['relevance_new'].astype(int)
y_pred = eval_data['predicted_relevance'].astype(int) #eval_data['predicted_relevance'].astype(int)

# 3. Считаем метрики
acc = accuracy_score(y_true, y_pred)
f1  = f1_score(y_true, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.6491
F1-score: 0.6094


# Итоги

Использование более глубокой модели приносит результат, но непропорционально затраченнным ресурсам компьютера. 

Считаю что бейзлайн вышел на ура - дешевая опенсорс модель локально раскидала наш датасет с очень хорошими показателями метрик за "5 строчек кода".

|Metrics|GPT OSS 20b|GPT OSS 120b|Agentic:20b|
|-|-|-|-|
|Accuracy:| 0.7160| 0.7480| 0.7020|
|F1-score:| 0.7617| 0.7907| 0.7306|

# Экспорт

In [ ]:
eval_data.to_excel("baseline/eval_data.xlsx")